Import Libraries

In [2]:
import pandas as pd
import numpy as np

### Warnings
import warnings

### Text Preprocessing and Natural Language Processing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import re
import spacy
from wordcloud import WordCloud

# For Fuzzy matching techniques
from rapidfuzz import process

# For parallel processing
import multiprocessing
from processing import find_best_match, process_batch

Import TMDB Dataset

In [119]:
pd.set_option('display.max_columns', None)

# Load in TMDB dataset
tmdb_df = pd.read_csv('TMDB_movie_dataset_v11.csv') 

# Drop Duplicates
tmdb_df = tmdb_df.drop_duplicates()

Creation of primary key in TMDB dataset (In order to join with the movielens dataset)

In [120]:
# Cleaning up the release date column to coerce problematic values to NaT, and ensure dates are in proper format
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'], errors='coerce')

# Creating new "year" (string) column and extracting year from the "release_date" field
# Changed .replace('nan', '') to .replace('<NA>', '')
tmdb_df['year'] = tmdb_df['release_date'].dt.year.astype('Int64').astype(str).replace('<NA>', '')

# Clean up movie titles to remove leading and trailing whitespace if any
tmdb_df['title'] = tmdb_df['title'].str.strip()

# Create a new column called title_year, combining title and year. If year is available, then create a new field combining title and year
tmdb_df['title_year'] = tmdb_df.apply(lambda x: f"{x['title']} ({x['year']})" if x['year'] else x['title'], axis=1)

Import MovieLens Datasets

In [121]:
# Load MovieLens Movies dataset
df_movies = pd.read_csv('ml-1m/movies.csv', sep='::', engine='python', 
                        encoding='ISO-8859-1', header=None, names=['MovieID', 'Title', 'Genres'])

# Clean up movie titles to remove leading and trailing whitespace
df_movies['Title'] = df_movies['Title'].str.strip()

# Load MovieLens Ratings dataset
df_ratings = pd.read_csv('ml-1m/ratings.csv', sep='::', engine='python', 
                         encoding='ISO-8859-1', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

# Load MovieLens Users dataset
df_users = pd.read_csv('ml-1m/users.csv', sep='::', engine='python', 
                       encoding='ISO-8859-1', header=None, names=['UserID', 'Gender', 'Age', 'Occupation','ZipCode'])

In [122]:
# Merge MovieLens Dataset into one dataframe (from three)
df_ml_combined = pd.merge(pd.merge(df_ratings, df_movies, on='MovieID'), df_users, on='UserID')

Truncate TMDB Dataset to show movies in Year 2000 and before only

In [123]:
# Filter out TMDB dataset for movies released in year 2000 and before
truncated_tmdb_df = tmdb_df[tmdb_df['release_date'].dt.year.astype('Int64') <= 2000]

Handle duplicates in the TMDB dateset involving the key 'title_year'

In [124]:
# Extract out rows from TMDB dataset with duplicate keys as they are non-unique
duplicates_tmdb = truncated_tmdb_df[truncated_tmdb_df['title_year'].duplicated(keep=False)]

# Get unique title-year keys from duplicates
unique_duplicated_title_years = duplicates_tmdb['title_year'].unique()

# Filter Combined MovieLens dataset to exclude rows with the duplicated keys from the analysis
filtered_df_ml_combined = df_ml_combined[~df_ml_combined['Title'].isin(unique_duplicated_title_years)]

Fixing Articles which appear at the end of movie titles in the filtered_df_ml_combined dataset

In [126]:
def format_title(title):
    # Split the title into words
    words = title.split()

    # Check if the last word is "A", "An", or "The"
    if len(words) >= 3 and words[-2].lower() in ['a', 'an', 'the'] and words[-3][-1] == ",":
        # Move the article to the beginning of the title
        formatted_title = f"{words[-2]} {' '.join(words[:-3])} {words[-3][:-1]} {words[-1]}"
    else:
        # Keep the title unchanged
        formatted_title = title

    return formatted_title

In [127]:
# Adjusting Movie Titles in correct sequence in the movielens dataset
filtered_df_ml_combined["Title"] = filtered_df_ml_combined["Title"].apply(format_title)

Combining both MovieLens and TMDB datasets (First iteration)

In [129]:
# Merge the 2 datasets using left-join (This will fix the `article` problem also) 
merged_df = pd.merge(filtered_df_ml_combined, truncated_tmdb_df, left_on='Title', right_on='title_year', how='left') 

Fixing discrepancies in movie release year

In [128]:
# Create new `Movie_Title` column and removing the year from the `Title` column
filtered_df_ml_combined["Movie_Title"] = filtered_df_ml_combined["Title"].str[:-7].str.strip()

# Create new `Movie_Year` column
filtered_df_ml_combined["Movie_Year"] = filtered_df_ml_combined["Title"].str[-5:-1].str.strip().astype(int)

In [181]:
# Extracting unsucessful merges and put in same format as filtered_df_ml_combined
unsuccessful_merge = merged_df[merged_df['title_year'].isnull()].iloc[:,:12]

#Retain original index
unsuccessful_merge["original_index"] = unsuccessful_merge.index

Combining both MovieLens and TMDB datasets (Second iteration) --> Fix Movie Release Year error in MovieLens dataset

In [191]:
# Merge the 2 datasets using left-join (second iteration) using movie-titles as keys 
temp_merged_df = pd.merge(unsuccessful_merge, truncated_tmdb_df, left_on='Movie_Title', right_on='title', how='left') 

# Only keep those successful merges
temp_merged_df = temp_merged_df[temp_merged_df["title"].notnull()]

# Convert year to integer for comparison
temp_merged_df["year"] = temp_merged_df["year"].astype(int)

# Create a new column and check if the movie years have a maximum absolute 1 year difference (To handle computational errors in the movielens dataset)
temp_merged_df['one_year_difference'] = abs(temp_merged_df['Movie_Year'] - temp_merged_df['year']) <= 1

# Only keep those rows where 1 year difference is TRUE
temp_merged_df = temp_merged_df[temp_merged_df['one_year_difference'] == True]

# Drop the 1 year difference column now
temp_merged_df.drop(columns=["one_year_difference"], inplace=True)

# Update Indexes to correspond to original merged_df
temp_merged_df.set_index("original_index", inplace= True)

In [199]:
# Update merged_df
merged_df = merged_df.combine_first(temp_merged_df)

Fuzzy Matching Techniques for string matching (Non Exact Match)  --> Third Iteration

In [210]:
# Filter out rows where the merge was unsuccessful, and put in same format as filtered_df_ml_combined
unmerged_v2 = merged_df[merged_df['title_year'].isnull()].iloc[:, :12]

# Keep original index from merged_df
unmerged_v2["original_index"] = unmerged_v2.index

In [209]:
unmerged_v2

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,ZipCode,Movie_Title,Movie_Year,original_index
43,1,2692,4,978301570,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance,F,1,10,48067,Run Lola Run (Lola rennt),1998,43
44,1,260,4,978300760,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,F,1,10,48067,Star Wars: Episode IV - A New Hope,1977,44
54,2,3068,4,978299000,The Verdict (1982),Drama,M,56,16,70072,The Verdict,1982,54
55,2,1537,4,978299620,Shall We Dance? (Shall We Dansu?) (1996),Comedy,M,56,16,70072,Shall We Dance? (Shall We Dansu?),1996,55
57,2,2194,4,978299297,The Untouchables (1987),Action|Crime|Drama,M,56,16,70072,The Untouchables,1987,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994407,6040,3703,4,964828575,Mad Max 2 (a.k.a. The Road Warrior) (1981),Action|Sci-Fi,M,25,6,11106,Mad Max 2 (a.k.a. The Road Warrior),1981,994407
994414,6040,2794,1,956716438,European Vacation (1985),Comedy,M,25,6,11106,European Vacation,1985,994414
994421,6040,2019,5,956703977,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama,M,25,6,11106,Seven Samurai (The Magnificent Seven) (Shichin...,1954,994421
994425,6040,549,4,956704746,Thirty-Two Short Films About Glenn Gould (1993),Documentary,M,25,6,11106,Thirty-Two Short Films About Glenn Gould,1993,994425


In [ ]:
# Function to perform similarity checking on a batch of data
def process_batch_parallel(batch, target_string):
    return [process.extractOne(target_string, [string])[1] for string in batch]

In [ ]:
from rapidfuzz import process

# Define a function to find the best match in the TMDB dataset for each title in MovieLens dataset
def find_best_match(title, choices):
    return process.extractOne(title, choices, score_cutoff=75)[0]

# Processing Function
def process_batch(batch):
    # Initialize empty list to store index of each row and its correspinding best match
    batch_matches = []
    
    for index, row in batch[0].iterrows():
        query_title = row['Title']
        # For unsuccessful row, find best match to title in TMDB dataset and change title to that. Use 75% Similarity Score threshold.
        best_match = find_best_match(query_title, batch[1])
        batch_matches.append((index, best_match))
        
    return batch_matches

In [16]:
# Parallel Function
def parallel_match(rows_to_match, trun_unique_tmdb_title_years):
    # Perform parallel processing on batches
    batch_results = pool.map(process_batch, [(batch, trun_unique_tmdb_title_years) for batch in batches])

    # Update 'Title' column in the MovieLens dataset with the best matches
    for batch_matches in batch_results:
        for index, best_match in batch_matches:
            filtered_df_ml_combined.at[index, 'Title'] = best_match
            
            
if __name__ == "__main__":
    batch_size = 1000
    batches = [unmerged_v2[i:i+batch_size] for i in range(0, len(data), batch_size)]
    
    # Get unique title_years from TMDB dataset for matching
    trun_unique_tmdb_title_years = truncated_tmdb_df['title_year'].unique()

    # Extract out rows which are unsuccessful in merging
    rows_to_match = merged_df[merged_df['title_year'].isnull()].iloc[0:100,:]
    
    parallel_match(rows_to_match, trun_unique_tmdb_title_years)

In [220]:
truncated_tmdb_df['title_year'].unique()

array(['Fight Club (1999)', 'Pulp Fiction (1994)', 'Forrest Gump (1994)',
       ..., 'Dead Kennedys: Live in Vienna (1982)',
       'Fuck You, Purdue (1987)', 'These Are the Rules (1983)'],
      dtype=object)